In [52]:
# 需要先匯入 By 這個類
from selenium.webdriver.common.by import By
from selenium import webdriver

url = "https://www.104.com.tw/"
driver = webdriver.Chrome()
driver.get(url)


In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
from collections import defaultdict
from pprint import pprint

# 創建一個 Options 物件
chrome_options = Options()

chrome_options.add_argument("--start-maximized") # Chrome 瀏覽器在啟動時最大化視窗
chrome_options.add_argument("--incognito") # 無痕模式
chrome_options.add_argument("--disable-popup-blocking") # 停用 Chrome 的彈窗阻擋功能。
    
service = Service('./chromedriver.exe')  # chromedriver 路徑

# 建立 WebDriver 物件
driver = webdriver.Chrome(service=service, options=chrome_options)

# 進入目標網站
driver.get('https://www.twse.com.tw/zh/trading/historical/stock-day-avg.html')

stock_price_dict = defaultdict(dict)
stock_date_dict = defaultdict(dict)
stock_code_list = ['00934', '00935', '00936']
# stock_code_list = ['00934']


for stock_code in stock_code_list:    
    stock_input = driver.find_element(By.CLASS_NAME, "stock-code-autocomplete")
    stock_input.send_keys(stock_code)

    el = driver.find_element(By.NAME, "date")
    driver.execute_script("arguments[0].value='20251001';", el)

    # 定位按鈕並點擊
    button_element = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/main/form/div/div[1]/div[3]/button")
    button_element.click()

    time.sleep(2)

    price_elements_list = driver.find_elements(By.CSS_SELECTOR, "tbody.is-last-page > tr")

    for element in price_elements_list:
        date_and_price = element.find_elements(By.TAG_NAME, 'td')

        date = date_and_price[0].text
        price = date_and_price[1].text
        if date == '月平均收盤價':
            continue

        stock_price_dict[stock_code][date] = price
        stock_date_dict[date][stock_code] = price
    
    stock_input.clear()

In [54]:
print(dict(stock_date_dict))

{'114/10/01': {'00934': '19.72', '00935': '26.14', '00936': '15.70'}, '114/10/02': {'00934': '19.83', '00935': '26.55', '00936': '15.73'}, '114/10/03': {'00934': '19.95', '00935': '27.12', '00936': '15.73'}, '114/10/07': {'00934': '20.30', '00935': '27.70', '00936': '15.90'}, '114/10/08': {'00934': '20.35', '00935': '27.62', '00936': '15.90'}, '114/10/09': {'00934': '20.31', '00935': '27.89', '00936': '15.87'}, '114/10/13': {'00934': '20.10', '00935': '27.54', '00936': '15.62'}, '114/10/14': {'00934': '19.95', '00935': '27.12', '00936': '15.47'}, '114/10/15': {'00934': '19.98', '00935': '27.64', '00936': '15.53'}, '114/10/16': {'00934': '20.28', '00935': '28.16', '00936': '15.74'}, '114/10/17': {'00934': '20.22', '00935': '27.80', '00936': '15.61'}, '114/10/20': {'00934': '20.38', '00935': '28.19', '00936': '15.62'}, '114/10/21': {'00934': '20.48', '00935': '28.37', '00936': '15.75'}, '114/10/22': {'00934': '20.56', '00935': '28.18', '00936': '15.83'}, '114/10/23': {'00934': '20.66', '

In [57]:

x = defaultdict(dict)

for date, price_dict in stock_date_dict.items():
    for k, v in price_dict.items():
        if x[k] == {}:
            x[k] = {
                "date": [date],
                "price": [v]
            }
        else:
            x[k]['date'].append(date)
            x[k]['price'].append(v)


In [58]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from datetime import datetime
from bokeh.palettes import Category10
import random
output_notebook()  # 若在 Jupyter Notebook 使用

p = figure(
    x_axis_type="datetime",
    width=600,
    height=400,
    title="股票價格折線圖"
)

for key, value in x.items():
    x[key]["date"] = [str(int(date[:3]) + 1911) + date[3:] for date in x[key]["date"]]

    x[key]["date"] = [datetime.strptime(date, "%Y/%m/%d") for date in x[key]["date"]]

    color = random.choice(Category10[10])  # 10 個漂亮顏色隨便挑

    p.line(x[key]["date"], x[key]["price"], line_width=2, color=color, legend_label=f"{key}")
    p.circle(x[key]["date"], x[key]["price"], size=6, color=color)


p.xaxis.axis_label = "日期"
p.yaxis.axis_label = "價格"

show(p)


Loading BokehJS ...